In [1]:
from PLSR_class import PLSRModel
import sys
sys.path.append('../baseline_for_training')
from Dataset import Dataset
import seaborn as sns
import pandas as pd

In [2]:
train_file_name = 'train_data.parquet'
validation_file_name = 'validation_data.parquet'
test_file_name = 'test_data.parquet'


dataset = Dataset(train_file_name,validation_file_name,test_file_name)

In [3]:
is_multi_output = True
PLSR_Tuning = True
param_grid = {'n_components': [i for i in range(1,10)]}
multi_PLSR = PLSRModel(dataset, param_grid, is_multi_output)

# Tuning Number of Components

In [4]:
from Training_and_Tuning import hyperParameterTuning

rmses = hyperParameterTuning(multi_PLSR,PLSR_Tuning = PLSR_Tuning)

Optimizing Number of Components: 100%|██████████| 9/9 [00:01<00:00,  4.64it/s]


In [5]:
best_n_components = sorted(rmses['Avg_RMSE'], key = lambda x:x[1])[0][0]['n_components']
print('Best n_components:', best_n_components)

Best n_components: 9


# Training with CV10

In [6]:
from Training_and_Tuning import CV10

multi_PLSR.model.estimator.set_params(n_components = best_n_components)

rmses = CV10(multi_PLSR)

Cross Validation: 100%|██████████| 10/10 [00:03<00:00,  3.11it/s]
